<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/5%20Procesado%20del%20lenguaje%20natural/Sesion%203/NLP_12_Bu%CC%81squeda_LSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsqueda de texto (*information retrieval*)
Vamos a usar el algoritmo LSI para realizar una búsqueda indexada de textos similares.
### Cargamos librerías

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import warnings

# Gensim
import gensim
import gensim.corpora as corpora

from gensim.models import TfidfModel, LsiModel
warnings.filterwarnings('ignore')

# spacy para lematizar
import spacy

Utilizamos un generador para obtener los documentos del Corpus línea a línea desde el archivo del conjunto de ejemplo y convertirlos en un listado de tokens.

In [ ]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner'])
stop_words = [word.text for word in nlp.vocab if word.is_stop] #listado de stop-words

def lemmatize_doc(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Función que devuelve el lema de una string,
    excluyendo las palabras cuyo POS_TAG no está en la lista"""
    text_out = [token.lemma_.lower() for token in nlp(text) if token.pos_ in allowed_postags and len(token.lemma_)>3]
    return text_out

class PreprocesaArchivo(object):
    """Pre-procesa un archivo de texto línea a línea
    Entrada: nombre del archivo de texto a procesar (string)
    Salida: iterador sobre cada línea normalizado (lista de tokens)"""
    def __init__(self, filename):
        self.filename = filename
 
    def __iter__(self):
        with open(self.filename) as f:
            for line in f:
                yield lemmatize_doc(line)

In [ ]:
data_dir = '{}'.format(os.sep).join([gensim.__path__[0], 'test', 'test_data'])
lee_data_file = data_dir + os.sep + 'lee_background.cor'

In [ ]:
texto=PreprocesaArchivo(lee_data_file)

### Creamos bigramas y trigramas
Creamos un modelo para las palabras más frecuentes como bigrama o trigrama para considerar estos tokens juntos en lugar de separados.

In [ ]:
#creamos bigramas y trigramas
bigram = gensim.models.Phrases(texto, min_count=5, threshold=50) # higher threshold fewer phrases.
#optimizamos una vez entreando
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram_mod[texto], min_count=5, threshold=50)  
trigram_mod = gensim.models.phrases.Phraser(trigram)

def make_trigrams(text):
    '''Devuelve un doc convertido en trigramas según el
    modelo trigram_mod. La entrada tiene que ser una lista
    de de tokens'''
    return trigram_mod[bigram_mod[text]]

class TrigramCorpus(object):
    """Pre-procesa un archivo de texto línea a línea
    Entrada: nombre del archivo de texto a procesar (string)
    Salida: iterador sobre cada línea normalizado (lista de tokens)"""
    def __init__(self, corpus):
        self.corpus = corpus
 
    def __iter__(self):
        for t in self.corpus:
            yield make_trigrams(t)

Transformamos el corpus de texto con el modelo de trigramas. Creamos un `generador` para no cargar todo el corpus procesado en memoria.

In [ ]:
textos_trigramas = TrigramCorpus(texto) #aplica modelo trigramas

### Creamos el diccionario y el corpus para Topic Modeling
Las dos entradas para el modelo LDA son un diccionario (id2word) y un corpus de `gensim`.  

In [ ]:
#para no tener que cargar todo el corpus en memoria creamos un streamer
class BOW_Corpus(object):
    """
    Iterable: en cada iteración devuelve el vector bag-of-words
    del siguiente documento en el corpus.
    
    Procesa un documento cada vez usando un generator, así
    nunca carga el corpus entero en RAM.
    """
    def __init__(self, corpus):
        self.corpus = corpus
        #crea el diccionario = mapeo de documentos a sparse vectors
        self.diccionario = gensim.corpora.Dictionary(corpus)
 
    def __iter__(self):
        """
        __iter__ es un generator => TxtSubdirsCorpus es un streamed iterable.
        """
        for tokens in self.corpus:
            # transforma cada doc (lista de tokens) en un vector sparse uno a uno
            yield self.diccionario.doc2bow(tokens)

In [ ]:
corpus_bow = BOW_Corpus(textos_trigramas)

Recuerda que en el modelo BoW de `gensim` el primer elemento de cada tupla es el ID del término en el diccionario, y el segundo su frecuencia en el doc.  
`diccionario[ID]` devuelve el término con índice ID en el vocabulario:

In [ ]:
len(corpus_bow.diccionario.token2id)

3893

## Topic modeling

### Modelo LSI
Este modelo ordena los temas y saca un listado ordenado. Hay que especificar el número de topics.  
Este modelo se calcula a partir de la matriz TF-IDF.

In [ ]:
modelo_tfidf = TfidfModel(corpus_bow)
corpus_tfidf = modelo_tfidf[corpus_bow]

In [ ]:
lsimodel = LsiModel(corpus=corpus_tfidf, num_topics=100, id2word=corpus_bow.diccionario)

In [ ]:
for c in corpus_tfidf:
    print(lsimodel[c])
    break

[(0, -0.2251407755258509), (1, -0.05589503255567473), (2, -0.002034551814245636), (3, -0.3593856462694006), (4, -0.28754330493655716), (5, 0.08107965643463073), (6, -0.080779156190972), (7, 0.1837984676332464), (8, 0.10152403453856364), (9, -0.08102866088669473), (10, -0.11996943236579523), (11, -0.0060285766977523445), (12, -0.029952780313636247), (13, -0.027187161104453246), (14, -0.07220274933336726), (15, -0.11674557418450958), (16, -0.06134103094887532), (17, -0.06612417144054032), (18, -0.12308969044055872), (19, 0.03399998170571504), (20, 0.005729132412297129), (21, 0.07489768934899206), (22, 0.0896517880076868), (23, -0.04938688915250265), (24, -0.03893218375433099), (25, -0.050089719015085014), (26, 0.0045414665732406625), (27, 0.05036297587865491), (28, 0.03238253066154584), (29, 0.08754126488999821), (30, -0.02747752826308267), (31, 0.040902897373087485), (32, 0.03684836491905905), (33, 0.019704835036649954), (34, 0.005582368474545655), (35, 0.06013677329826353), (36, 0.0289

##  Búsqueda de documentos por temática (*information retrieval*)
Para buscar los documentos más similares a un documento dado, hay que trabajar con el modelo *space vector* generado por el algoritmo LSI. Primero, generamos una matriz LSI para todos los documentos del corpus. Para buscar el documento más parecido a un nuevo texto, calculamos su vector LSI y buscamos cuál es el más cercano dentro de la matriz LSI del corpus.

In [ ]:
#creamos un índice de similitud entre los documentos del corpus
from gensim.similarities import MatrixSimilarity

#creamos corpus transformado
lsi_corpus = lsimodel[corpus_tfidf]

In [ ]:
lsi_corpus

In [ ]:
#creamos índice
lsi_corpus = list(lsi_corpus) #hay que pasarlo a una lista en memoria
index = MatrixSimilarity(lsi_corpus)

In [ ]:
index

Podemos ver la similitud de cualquier documento del corpus al resto de documentos

Esta matriz de similaridad usa (300 vectores de 100 componentes, con el grado de pertenencia de acda documento a cada una de las dimensiones). 

Guarda la similitud de cada documento con cada uno de los 299 restantes, utilizando la similitud coseno de los vectores de documentos asociados a cada documento (recordemos que cada documento lo hemos 'representado' como un conjunto de 100 valores/componentes)

Por ejemplo, tal y como se puede apreciar en la celda siguiente en la que se ven las similaridades con el documento 1, la similitud entre el documento 0 y el 1 es de alrededor de 0.1, la similitud entre el documento 1 y el 1 es de prácticamente 1, la similitud entre el documento 1 y 2 es de en torno a 0.015

In [ ]:
sims = index[lsi_corpus[1]]
print(list(enumerate(sims)))

[(0, 0.096995465), (1, 0.99999994), (2, 0.016084313), (3, 0.036357336), (4, -0.005169048), (5, 0.035257604), (6, 0.07894909), (7, -0.019114595), (8, 0.048595857), (9, 0.10794885), (10, 0.067132495), (11, 0.015211977), (12, 0.5141053), (13, -0.022404313), (14, 0.056497663), (15, 0.0024327785), (16, 0.0017246548), (17, 0.026247222), (18, 0.016068885), (19, -0.0411802), (20, -0.037240278), (21, 0.019896697), (22, 0.009519938), (23, 0.012997538), (24, 0.077828035), (25, 0.035765894), (26, 0.4104902), (27, 0.016949251), (28, 0.03527139), (29, 0.029354021), (30, -0.015645945), (31, 0.10353141), (32, -0.03771535), (33, 0.1224274), (34, 0.6077539), (35, 0.06483999), (36, 0.14475338), (37, 0.058779977), (38, 0.039481375), (39, 0.003149828), (40, 0.09460867), (41, 0.26310873), (42, 0.017722148), (43, 0.023563396), (44, 0.049257606), (45, 0.03996374), (46, 0.043322645), (47, 0.08864278), (48, 0.092150636), (49, 0.037699647), (50, 0.09577018), (51, 0.07028355), (52, 0.014994817), (53, 0.022919115)

In [ ]:
#nos quedamos con los 10 primeros
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims_sorted[:10])

[(1, 0.99999994), (143, 0.67422736), (34, 0.6077539), (12, 0.5141053), (85, 0.5060218), (26, 0.4104902), (116, 0.320574), (227, 0.31225094), (220, 0.30432057), (141, 0.28305617)]


También podemos calcular el documento más similar dentro del corpus a un nuevo documento calculando primero su matriz TF-IDF/BoW y luego transformando a matriz LSI

In [ ]:
new_doc = "the new pakistan government falled in the terrorist attack by the islamic group hamas"
texto_lemmatizados = lemmatize_doc(new_doc)
texto = make_trigrams(texto_lemmatizados)
corpus_new = corpus_bow.diccionario.doc2bow(texto)
lsi_corpus_new = lsimodel[corpus_new]

In [ ]:
texto

['pakistan',
 'government',
 'fall',
 'terrorist',
 'attack',
 'islamic',
 'group',
 'hamas']

In [ ]:
print(lsi_corpus_new)

[(0, -0.3613815479095896), (1, 0.19534752064686686), (2, -0.029855132428111643), (3, 0.01212041010484912), (4, 0.10984006678292624), (5, -0.0886267872635145), (6, 0.06598682864671415), (7, 0.05025922017894794), (8, 0.060260259573719804), (9, 0.12579025371953695), (10, 0.11260553943861919), (11, -0.05871609642986213), (12, -0.1741038616392151), (13, 0.028063100517521047), (14, 0.19478652390968715), (15, -0.02756769221077642), (16, -0.03704635354393259), (17, -0.14737547872746043), (18, -0.07309591948875903), (19, -0.05835029377225188), (20, 0.1924322965556895), (21, 0.0651554227036181), (22, 0.004326856838880198), (23, -0.027770096801394376), (24, 0.10000302434515991), (25, -0.07779878817932762), (26, 0.005529697593091548), (27, -0.07859458931789867), (28, -0.018084791489868492), (29, -0.06634210806356522), (30, -0.01695578625547719), (31, -0.055814108277498886), (32, 0.07323225219422785), (33, 0.15452653670404437), (34, -0.057824185136652814), (35, 0.1787275396770587), (36, -0.00256357

Ahora buscamos en el índice cuáles son los documentos más parecidos dentro del corpus al nuevo documento:

In [ ]:
sims = index[lsi_corpus_new]
sims_sorted = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims_sorted[:10])

[(143, 0.5675734), (267, 0.46883836), (160, 0.4677524), (60, 0.43944794), (116, 0.4372841), (75, 0.41249728), (1, 0.39473557), (217, 0.37357435), (223, 0.36711973), (197, 0.3669526)]


El texto del documento más cercano es:

In [ ]:
with open(lee_data_file) as f:
    textos = f.readlines()
print(textos[sims_sorted[0][0]])

Kashmiri militant groups denied involvement in Thursday's attack on the Indian Parliament, accusing Indian intelligence instead. "We want to make it clear that Kashmiris have no connection with this attack," said the Muttahida Jihad Council (MJC), an alliance of 18 groups fighting Indian rule in Kashmir. "We believe it was carried out by Indian intelligence agencies to achieve their motives about the Kashmir issue," the groups added in a statement. The attack on the Parliament building in New Delhi left at least 12 dead. The Indian authorities have not said who they believe was behind the killings. But the Kashmiri groups accused the Indian Government of masterminding the attack in a bid to divert attention from what they called increasing international pressure over Kashmir. 

